In [64]:
import pandas as pd
import numpy as np
import os
import math

data_path = '/Users/tyamgin/Downloads/vk2/'

In [81]:
pd.read_csv(os.path.join(data_path, 'result.csv'))

,uid,age
0,356,35
1,26356,35
2,97,35
3,319,35
4,30288,35
...,...,...
29464,33331,35
29465,119788,35
29466,97581,35
29467,104038,35


In [14]:
friends = pd.read_csv(os.path.join(data_path, 'friends.csv'))
friends

,uid,fuid
0,105728,104361
1,2026,38044
2,107090,6391
3,88157,6391
4,49843,6391
...,...,...
4109872,23805,36888
4109873,68932,51548
4109874,112906,5612
4109875,73731,35997


In [15]:
train = pd.read_csv(os.path.join(data_path, 'train.csv'))
train

,uid,age,registered_year
0,85788,32,2020
1,19171,30,2009
2,7780,51,2012
3,14998,17,2019
4,94913,33,2010
...,...,...,...
29464,12575,34,2011
29465,96062,39,2012
29466,33676,44,2012
29467,24043,38,2011


In [16]:
train_edu = pd.read_csv(os.path.join(data_path, 'trainEducationFeatures.csv'))
train_edu

,uid,school_education,graduation_1,graduation_2,graduation_3,graduation_4,graduation_5,graduation_6,graduation_7
0,356,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,26356,NaN,NaN,NaN,NaN,NaN,2006.0,NaN,NaN
2,97,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,319,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,30288,2008.0,NaN,NaN,NaN,NaN,2013.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...
29464,33331,1987.0,NaN,NaN,NaN,NaN,1991.0,NaN,NaN
29465,119788,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29466,97581,2002.0,NaN,NaN,NaN,NaN,2007.0,NaN,NaN
29467,104038,1969.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
train_groups = pd.read_csv(os.path.join(data_path, 'trainGroups.csv'))
train_groups

,uid,gid
0,34673,110624
1,119303,108687
2,89035,234084
3,7322,351543
4,102327,304142
...,...,...
1086086,41186,90661
1086087,80927,348122
1086088,68207,213837
1086089,112909,342369


In [23]:
train_edu[~train_edu.school_education.isna()]

,uid,school_education,graduation_1,graduation_2,graduation_3,graduation_4,graduation_5,graduation_6,graduation_7
4,30288,2008.0,NaN,NaN,NaN,NaN,2013.0,NaN,NaN
9,25275,2007.0,NaN,NaN,NaN,NaN,NaN,2011.0,NaN
14,116156,2008.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,10794,2006.0,NaN,NaN,NaN,NaN,NaN,NaN,2014.0
17,99780,1995.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
29463,90272,2010.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29464,33331,1987.0,NaN,NaN,NaN,NaN,1991.0,NaN,NaN
29466,97581,2002.0,NaN,NaN,NaN,NaN,2007.0,NaN,NaN
29467,104038,1969.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [97]:
gg = train_edu[~train_edu.graduation_1.isna()].set_index('uid').join(train.set_index('uid'), how='left')
gg

,school_education,graduation_1,graduation_2,graduation_3,graduation_4,graduation_5,graduation_6,graduation_7,res,age,registered_year
uid,,,,,,,,,,,
54766,NaN,2008.0,NaN,NaN,NaN,NaN,NaN,NaN,35.0,33,2013
117326,2010.0,2016.0,NaN,NaN,NaN,NaN,NaN,NaN,28.0,28,2008
113602,NaN,2018.0,NaN,NaN,NaN,NaN,NaN,NaN,35.0,24,2010
64060,NaN,2019.0,NaN,NaN,NaN,NaN,NaN,NaN,35.0,22,2019
78026,NaN,2017.0,NaN,NaN,NaN,NaN,NaN,NaN,35.0,28,2009
3666,NaN,2011.0,NaN,NaN,NaN,NaN,NaN,NaN,35.0,28,2009
65032,2006.0,2012.0,NaN,NaN,NaN,NaN,NaN,NaN,32.0,33,2008
71788,NaN,2022.0,NaN,NaN,NaN,NaN,NaN,NaN,35.0,22,2020
106944,NaN,2008.0,NaN,NaN,NaN,NaN,NaN,NaN,35.0,43,2010


In [45]:
def rmse(x, y):
    n = len(x)
    return math.sqrt(1.0 * sum((x - y)**2) / n)

In [103]:
for grad in ('school_education', 'graduation_1', 'graduation_2',  'graduation_3', 'graduation_4',
             'graduation_5', 'graduation_6', 'graduation_7'):
    gg = train_edu[~train_edu[grad].isna()].set_index('uid').join(train.set_index('uid'), how='left')
    best_age = -1
    min_rmse = 1e10
    for age in range(10,  80):
        r = rmse(2021 - gg[grad] + age, gg.age)
        if r < min_rmse:
            best_age = age
            min_rmse = r
    print(f"'{grad}': {best_age},  # {min_rmse}")
        

'school_education': 17,  # 4.903859550418915
'graduation_1': 25,  # 5.590608371258384
'graduation_2': 24,  # 4.912185675064848
'graduation_3': 24,  # 4.905354217587146
'graduation_4': 25,  # 4.009603855360661
'graduation_5': 24,  # 5.55578449198568
'graduation_6': 25,  # 5.225676253624243
'graduation_7': 25,  # 7.671183665089072


In [46]:
rmse(2021 - gg.school_education + 17, gg.age)

4.903859550418915

In [121]:
def solve(edu):
    edu['res'] = 34
    dct = {
        'school_education': 17,  # 4.903859550418915
        'graduation_1': 25,  # 5.590608371258384
        'graduation_2': 24,  # 4.912185675064848
        'graduation_3': 24,  # 4.905354217587146
        'graduation_4': 25,  # 4.009603855360661
        'graduation_5': 24,  # 5.55578449198568
        'graduation_6': 25,  # 5.225676253624243
        'graduation_7': 25,  # 7.671183665089072
    }
    for grad, shift in dct.items():
        grad_idx = ~edu[grad].isna()
        edu.loc[grad_idx, 'res'] = 2021 - edu.loc[grad_idx, grad] + shift

    return edu[['uid', 'res']].set_index('uid')
    
res = solve(train_edu)
res

,res
uid,
356,34.0
26356,39.0
97,34.0
319,34.0
30288,32.0
...,...
33331,54.0
119788,34.0
97581,38.0


In [122]:
res_joined = res.join(train.set_index('uid'), how='left')
rmse(res_joined.res, res_joined.age)

11.47363369861934

In [ ]:
# 13.12131431455484 - 35
# 11.585301610692005 - 35 + school
# 11.512968371985176 - 35 + all
# 11.47363369861934 - 34 + all